In [16]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

def ETL_read():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    ratings = pd.read_csv(ratings_file, low_memory=False)
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)

    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file) 
    
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    
    # 5. Return the three DataFrames
    return wiki_movies_df, kaggle_metadata, ratings

# 6 Create the path to your file directory and variables for the three files. 
file_dir = 'C:/Users/Patri/Documents/GitHub/Movies-ETL/data'
# Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

# 7. Set the three variables in Step 6 equal to the function created in Step 1.
wiki_file, kaggle_file, ratings_file = ETL_read()

In [17]:
# 1. Add the clean movie function that takes in the argument, "movie".
alt_title_headers = ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional', 'Yale Romanization', 'Yiddish'
                    'Hanyu Pinyin', 'Simplified Chinese', 'Traditional Chinese']

# creating function to clean data
def clean_movie(movie):
    # sets local variable movie to be referenced within function so original data untouched
    movie = dict(movie) # non-destructive copy
    alt_titles = {} # empty dict
    
    for key in alt_title_headers:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)

    # adding alternatives title dict to movie object
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles   
        
        
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    # consolidating columns
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    return movie

In [18]:
# deliverable 2 code

# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def ETL_read():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    ratings = pd.read_csv(ratings_file, low_memory=False)
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)

    # 3. Open the read the Wikipedia data JSON file.
    with open(wiki_file, mode='r') as file:
        wiki_movies_raw = json.load(file) 
    
    # 4. Read in the raw wiki movie data as a Pandas DataFrame.
    #wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_wiki_movies = [clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    cleaned_wiki_df = pd.DataFrame(clean_wiki_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        cleaned_wiki_df['imdb_id'] = cleaned_wiki_df['imdb_link'].str.extract(r'(tt\d{7})')
        cleaned_wiki_df.drop_duplicates(subset='imdb_id', inplace=True)
    except NameError:
        print("Name does not exist!")
        
        
    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in cleaned_wiki_df.columns if cleaned_wiki_df[column].isnull().sum() < len(cleaned_wiki_df) * 0.9]
    wiki_movies_df = cleaned_wiki_df[wiki_columns_to_keep]  
    
    
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)


    # 2. Clean the Kaggle metadata.
    # dropping all adult titles from dataset
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')

     # creating boolean column 'video' and assigning it to series
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

        # using pandas to numeric to convert objects to num, use raise error flag to ensure imports are fine
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

        # converting object to datetime object
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])


        # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(cleaned_wiki_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

        # 4. Drop unnecessary columns from the merged DataFrame.
    movies_df.drop(columns=['title_wiki', 'Language','Production company(s)'], inplace=True)

        # 5. Add in the function to fill in the missing Kaggle data.
        # creating function that fills in missing data and drops redundant column
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
                            , axis=1)
        df.drop(columns=wiki_column, inplace=True)

        # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.

    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')

        # 7. Filter the movies DataFrame for specific columns.


        # 8. Rename the columns in the movies DataFrame.
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                           'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
                           'genres','original_language','overview','spoken_languages','Country',
                           'production_companies','production_countries','Distributor',
                           'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
                          ]]

        # renaming columns
    movies_df.rename({'id':'kaggle_id',
                          'title_kaggle':'title',
                          'url':'wikipedia_url',
                          'budget_kaggle':'budget',
                          'release_date_kaggle':'release_date',
                          'Country':'country',
                          'Distributor':'distributor',
                          'Producer(s)':'producers',
                          'Director':'director',
                          'Starring':'starring',
                          'Cinematography':'cinematography',
                          'Editor(s)':'editors',
                          'Writer(s)':'writers',
                          'Composer(s)':'composers',
                          'Based on':'based_on'
                         }, axis='columns', inplace=True)

        # 9. Transform and merge the ratings DataFrame.

        # pivoting the table to make movie id the index, counts of ratings the columns
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                        .rename({'userId':'count'}, axis=1) \
                        .pivot(index='movieId',columns='rating', values='count')

        # renaming columns
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

     
    
    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_1 = r'\$\d+\.?\d*\s*[mb]illion'
    
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_2 = r'\$\s*\d{1,3}(?:[,|.]\d{3})+(?!\s[mb]illi?on)'

    # 12. Add the parse_dollars function.
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
    
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_1}|{form_2})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    
    # converting lists to strings
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    
    # removing values between dollar sign and hyphens to find ranges
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    wiki_movies_df['budget'] = budget.str.extract(f'({form_1}|{form_2})', flags=re.IGNORECASE)[0].apply(parse_dollars)


    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    # parsing 4 forms
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    form1 = r'[January|February|March|April|May|June|July|August|September|October|November|December]\s*[1-31],\s*\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    f1_match = release_date.str.contains(form1, flags=re.IGNORECASE)
    
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({f1_match}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    
    # converting strings to numbers, and using coerce to set empty strings to NaN
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

    
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 



In [19]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 'C:/Users/Patri/Documents/GitHub/Movies-ETL/data'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [20]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = ETL_read()

KeyError: 'budget_kaggle'

In [ ]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [ ]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

In [ ]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [ ]:
# 15. Check the movies_df DataFrame. 
movies_df.head()